In [ ]:
import os
import json
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import librosa
import torch

# Genre mapping
genre_mapping = {
    0: "Electronic",
    1: "Rock",
    2: "Punk",
    3: "Experimental",
    4: "Hip-Hop",
    5: "Folk",
    6: "Chiptune / Glitch",
    7: "Instrumental",
    8: "Pop",
    9: "International",
}

# Load model and feature extractor using Facebook Wav2vec2 and gastondault Music-classifier
model = Wav2Vec2ForSequenceClassification.from_pretrained("gastonduault/music-classifier")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large")

# Function for preprocessing audio
def preprocess_audio(audio_path):
    try:
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)
        return feature_extractor(audio_array, sampling_rate=16000, return_tensors="pt", padding=True)
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

# Function for predicting genre
def predict_genre(audio_path):
    inputs = preprocess_audio(audio_path)
    if inputs is None:
        return None
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class = torch.argmax(logits, dim=-1).item()
        confidence = torch.softmax(logits, dim=-1)[0, predicted_class].item()
    return genre_mapping[predicted_class], confidence

# Directory of audio samples
audio_directory = "/content/U Got Me"  # Replace with your directory path
output_results = []

# Iterate over all .wav files in the directory
for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        audio_path = os.path.join(audio_directory, filename)
        print(f"Processing: {filename}")
        result = predict_genre(audio_path)
        if result:
            genre, confidence = result
            output_results.append({
                "filename": filename,
                "genre": genre,
                "confidence": f"{confidence:.2%}"
            })
            print(f"Predicted genre: {genre} (Confidence: {confidence:.2%})")

# Save results to a JSON file
output_file = os.path.join(audio_directory, "U Got Me_genre_predictions.json")
with open(output_file, "w") as f:
    json.dump(output_results, f, indent=4)

print(f"Processed {len(output_results)} files. Results saved to {output_file}.")


Processing: U Got Me_segment_2.wav
Predicted genre: Hip-Hop (Confidence: 74.19%)
Processing: U Got Me_segment_3.wav
Predicted genre: International (Confidence: 60.45%)
Processing: U Got Me_segment_6.wav
Predicted genre: International (Confidence: 65.25%)
Processing: U Got Me_segment_5.wav
Predicted genre: International (Confidence: 53.34%)
Processing: U Got Me_segment_4.wav
Predicted genre: Hip-Hop (Confidence: 60.64%)
Processing: U Got Me_segment_1.wav
Predicted genre: International (Confidence: 68.95%)
Processed 6 files. Results saved to /content/U Got Me/U Got Me_genre_predictions.json.
